In [3]:
import pandas as pd
import numpy as np
import random
import gc
from os import listdir
from xgboost import XGBClassifier
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import SGD, RMSprop
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier

Using Theano backend.


# Note types
- 0: nothing
- 1: step
- 2: hold start
- 3: hold/roll end
- 4: roll start
- M: mine

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [15]:
get_class_for_index([['111M']], 0)

[3, 6]

In [14]:
samples_back_included = 8
num_classes = 7
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, index):
    return beat_features[index] if index >= 0 else [0] * num_features

def get_class_for_index_expanded(notes, index):
    if index < 0:
        return [1, 0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    if steps == 0 and mines == 0 and holds == 0 and rolls == 0:
        return [1, 0, 0, 0, 0, 0, 0]
    steps += (holds + rolls)
    return [int(i) for i in [False, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]

def get_class_for_index(notes, index):
    classes_expanded = get_class_for_index_expanded(notes, index)
    return [i for i in range(7) if classes_expanded[i]]
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(X, y, key, is_full):
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            class_nums = get_class_for_index(notes, i)
            for class_num in class_nums:
                if is_full or not (class_num == 0 and random.randint(0, 3) > 0):
                    features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, i - j)]
                    features.extend([i % 48, get_beat_importance(i), i / 48, num_notes - i / 48])
                    X.append(features)
                    y.append(class_num)

# Total 243 songs
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
np.random.shuffle(songs_to_use)
def build_training_data(start, end, is_full):
    X = []
    y = []
    for song_data in songs_to_use[start:end]:
        get_features_for_song(X, y, song_data[0], is_full)
    return np.array(X), np.array(y)

## Final Models

#### max_depth = 3 (0.881996974281)
0.885299268107
CPU times: user 1h 55min 59s, sys: 33.4 s, total: 1h 56min 33s
Wall time: 31min 36s

#### max_depth = 5 (0.882895702581)
0.887660335563
CPU times: user 3h 8min 31s, sys: 35.2 s, total: 3h 9min 6s
Wall time: 51min 12s

#### max_depth=7 (0.883434939561)
0.896775328155
CPU times: user 4h 20min 18s, sys: 52 s, total: 4h 21min 10s
Wall time: 1h 10min 44s

#### max_depth=10 (0.882386423211)
0.916481448221
CPU times: user 6h 5min 3s, sys: 1min 37s, total: 6h 6min 40s
Wall time: 1h 40min 41s

In [3]:
xgb_clf = XGBClassifier(max_depth=7, min_child_weight=8, learning_rate=0.05, seed=0, n_estimators=100, subsample=0.80, colsample_bytree=0.80, objective="multi:softprob")
# 0.896775328155
# 0.883434939561

In [4]:
rf_clf = RandomForestClassifier(n_estimators = 100, max_features=200, min_samples_leaf=12, verbose=True)
# 0.942960202356
# 0.882506253651

In [5]:
sgd_clf = SGDClassifier(loss="modified_huber", n_iter=50)
# 0.815508609316
# 0.827428696147

In [6]:
%%time
X, y = build_training_data(0, 243, True)
gc.collect()

CPU times: user 1min 31s, sys: 40 s, total: 2min 11s
Wall time: 2min 26s


In [7]:
%%time
eclf = EnsembleVoteClassifier(clfs=[xgb_clf, rf_clf, sgd_clf], weights=[1, 1, 1], voting='soft')
eclf.fit(X, y)
print (eclf.score(X, y))
joblib.dump(eclf, 'models/song_class_eclf/clf.pkl')
gc.collect()

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed: 246.8min
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 507.0min finished
[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    9.9s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   21.4s finished


0.882169191936
CPU times: user 16h 6min 11s, sys: 3min 15s, total: 16h 9min 27s
Wall time: 10h 39min 33s


In [8]:
xgb_clf = XGBClassifier(max_depth=7, min_child_weight=8, learning_rate=0.05, seed=0, n_estimators=100, subsample=0.80, colsample_bytree=0.80, objective="multi:softprob")
rf_clf = RandomForestClassifier(n_estimators = 100, max_features=200, min_samples_leaf=12, verbose=True)
sgd_clf = SGDClassifier(loss="modified_huber", n_iter=50)

In [9]:
%%time
X, y = build_training_data(0, 243, False)
gc.collect()

CPU times: user 45.5 s, sys: 2.53 s, total: 48.1 s
Wall time: 50 s


In [10]:
%%time
eclf = EnsembleVoteClassifier(clfs=[xgb_clf, rf_clf, sgd_clf], weights=[1, 1, 1], voting='soft')
eclf.fit(X, y)
print (eclf.score(X, y))
joblib.dump(eclf, 'models/song_class_eclf_partial/clf.pkl')
gc.collect()

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed: 60.5min
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 123.1min finished
[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    3.4s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    7.0s finished


0.832920535706
CPU times: user 3h 59min 4s, sys: 27.1 s, total: 3h 59min 31s
Wall time: 2h 35min 5s


## Not Using

In [ ]:
%%time
model = Sequential()

model.add(Dense(512, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=50, batch_size=64, verbose=1)
print (model.evaluate(X_test, y_test, batch_size=64))
model.save('models/song_class_model.h5')
model = None
gc.collect()
# 0.884
# 0.86375278980600079